In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.applications.vgg19 import VGG19 
from keras.applications.mobilenet import MobileNet
from keras.applications.resnet50 import ResNet50

from keras.layers import Dense
from keras.models import Model
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.utils.vis_utils import plot_model

from keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix #classification_report
import numpy as np
import pandas as pd
import cv2

import matplotlib.pyplot as plt
%matplotlib inline  
# for jupyter notebook enviornment. 

import itertools  # for confusion matrix plot

from PIL import Image as pil_image

class_label = ['angry', 'happy','neutral']
n_class = len(class_label)

img_size = 48  # fer data size. 48 x 48
target_size = 48 #197 # minimum data size for specific net such as Inception, VGG, ResNet ...
#n_data = 1000  # to deal with memory issue..

epochs = 20  # n of times of training using entire data


def data_arrange(x_data,y_data):
    # data class re-arrange
    
    # Angry vs neutral case
    x_angry = x_data[y_data==0]
    x_happy = x_data[y_data==3]
    x_neutral = x_data[y_data==6]
    
    y_angry = y_data[y_data==0]
    y_happy = y_data[y_data==3]
    y_neutral = y_data[y_data==6]
    
    # number of happy samples are twice  
    # To avoid class distribution bias. use only 50% sample of happy class
    x_happy_use, x_no, y_happy_use, y_no = train_test_split(x_happy, y_happy, test_size = 0.5, shuffle = True, random_state=33)
    
    #print('Before normalize:{a}\n'.format(a= x_angry[0]))
    xx = np.concatenate((x_angry, x_happy_use, x_neutral),axis=0)/255.0 #concatenate & normalized
    yy = np.concatenate((y_angry, y_happy_use, y_neutral), axis=0)
    yy[yy==3]=1
    yy[yy==6]=2
    
    xx = xx.reshape(-1, img_size,img_size)
    xx = np.stack((xx,)*3, -1 )  # to make fake RGB channel
    yy = np_utils.to_categorical(yy, n_class)
    print('After normalize x:{a} y:{b}\n'.format(a= xx.shape, b=yy.shape))     

    return xx, yy

def sample_plot(x_test, y_test):
    x_angry = x_test[y_test==0]
    a_f = np.squeeze(x_angry[1])
    plt.imshow(a_f, cmap='gray')
    
def plot_hist(hist):
    plt.figure(0)
    fig, loss_ax = plt.subplots()
    acc_ax = loss_ax.twinx()

    loss_ax.plot(hist.history['loss'], 'y', label='train loss')
    loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
    loss_ax.set_xlabel('epoch')
    loss_ax.set_ylabel('loss')
    loss_ax.legend(loc='upper left')

    acc_ax.plot(hist.history['acc'], 'b', label='train acc')
    acc_ax.plot(hist.history['val_acc'], 'g', label='val acc')
    acc_ax.set_ylabel('accuracy')
    acc_ax.legend(loc='lower left')

    plt.show()
    #plt.savefig('loss_accuracy_plot')
    plt.close()

# Make and plot confusion matrix. To see the detailed imformation about TP, TN of each classes.    
def make_confusion_matrix(model, x, y, normalize = True):
    predicted = model.predict(x)

    pred_list = []; actual_list = []
    for i in predicted:
        pred_list.append(np.argmax(i))
    for i in y:
        actual_list.append(np.argmax(i))

    confusion_result = confusion_matrix(actual_list, pred_list)
    plot_confusion_matrix(confusion_result, classes = class_label, normalize = normalize, title = 'Confusion_matrix')
    return confusion_result

def plot_confusion_matrix(cm, classes,
                          normalize = False,
                          title = 'Confusion matrix',
                          cmap = plt.cm.Blues):  
    if normalize:
        cm = cm.astype('float') / cm.sum(axis = 1)[:, np.newaxis]
        print("normalized")
    else:
        print('without normalization')

    print(cm)
    plt.figure(1)
    plt.imshow(cm, interpolation='nearest', cmap = cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment = "center",
                 color = "white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    #plt.savefig('Confusion_matrix')
    


In [25]:
os.chdir('/Data/')
x_data = np.load('./x_data.npy')
y_data = np.load('./y_data.npy')
    
# 2. arrange the data. shape change, use specific class only, ...
x_data, y_data = data_arrange(x_data, y_data)

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2, shuffle = True, random_state=33)
print('Transfer learning.\n\n Model: \n')

# you can selecte max pooling instead avg
model_transfer = MobileNet(input_shape=(target_size, target_size, 3), alpha=1.0, depth_multiplier=1, dropout=1e-3, 
                                       include_top=False, weights=None, input_tensor=None, pooling='avg', classes=n_class)#n_class)

#model_transfer = VGG19(include_top = False, weights = 'imagenet',
                     #  input_tensor = None, input_shape = (target_size, target_size, 3), pooling = 'avg' , classes = n_class)

##### Add new output for fine-tuning
# you can add more layers, or another layers
x = model_transfer.output
x = Dense(2048, activation ='relu')(x)
predictions = Dense(n_class, activation ='softmax')(x)
model = Model(inputs = model_transfer.input, outputs = predictions)

##### Freeze all layers in the resnet. we just use the pretrained weight, during training, no back propagation will appear here.
# But you can selectively activate specific layer for training.
for layer in model_transfer.layers:
    layer.trainable = False

##### Start learning. same procedure.
model.compile(loss = categorical_crossentropy,
                  optimizer=Adam(lr = 0.001, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-7),
                  metrics=['accuracy'])


early_stopping = EarlyStopping(monitor = 'val_acc', min_delta = 0.001, patience = 10, 
                                       verbose = 1, mode = 'max')



After normalize x:(18589, 48, 48, 3) y:(18589, 3)

Transfer learning.

 Model: 



"\n##### Add new output for fine-tuning\n# you can add more layers, or another layers\nx = model_transfer.output\nx = Dense(2048, activation ='relu')(x)\npredictions = Dense(n_class, activation ='softmax')(x)\nmodel = Model(inputs = model_transfer.input, outputs = predictions)\n\n##### Freeze all layers in the resnet. we just use the pretrained weight, during training, no back propagation will appear here.\n# But you can selectively activate specific layer for training.\nfor layer in model_transfer.layers:\n    layer.trainable = False\n\n##### Start learning. same procedure.\nmodel.compile(loss = categorical_crossentropy,\n                  optimizer=Adam(lr = 0.001, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-7),\n                  metrics=['accuracy'])\n\n\nearly_stopping = EarlyStopping(monitor = 'val_acc', min_delta = 0.001, patience = 10, \n                                       verbose = 1, mode = 'max')\n\n"

In [26]:
model_transfer.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
__________

In [3]:
## Fit generator with augmentation

epochs = 20
batch_size = 32


### Augmentation options

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0)

datagen.fit(x_train)

# if you have powerful hardware like GPU, multicore, you can change workers, multiprocess option 
hist = model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch = len(x_train)/batch_size, epochs=epochs,
                        workers=1, validation_data=(x_test, y_test), use_multiprocessing=False, callbacks = [early_stopping] )


Epoch 1/20
280/464 [=================>............] - ETA: 2:33 - loss: 1.2171 - acc: 0.3761

KeyboardInterrupt: 

In [13]:
epochs = 20
hist = model.fit(x_train, y_train, 
                              validation_split = 0.2, 
                              shuffle = True, 
                              batch_size = 16, epochs = epochs, verbose = 1, 
                              callbacks = [early_stopping] )
        

Train on 11896 samples, validate on 2975 samples
Epoch 1/20
11896/11896 [==============================] - 236s 20ms/step - loss: 1.0257 - acc: 0.4738 - val_loss: 0.9842 - val_acc: 0.5150
Epoch 2/20
11896/11896 [==============================] - 234s 20ms/step - loss: 0.9625 - acc: 0.5319 - val_loss: 0.9651 - val_acc: 0.5291
Epoch 3/20
11896/11896 [==============================] - 238s 20ms/step - loss: 0.9378 - acc: 0.5563 - val_loss: 0.9423 - val_acc: 0.5429
Epoch 4/20
11896/11896 [==============================] - 234s 20ms/step - loss: 0.9231 - acc: 0.5590 - val_loss: 0.9315 - val_acc: 0.5576
Epoch 5/20
11896/11896 [==============================] - 255s 21ms/step - loss: 0.9123 - acc: 0.5678 - val_loss: 0.9311 - val_acc: 0.5613
Epoch 6/20
11896/11896 [==============================] - 235s 20ms/step - loss: 0.9034 - acc: 0.5767 - val_loss: 0.9295 - val_acc: 0.5529
Epoch 7/20
11896/11896 [==============================] - 233s 20ms/step - loss: 0.8971 - acc: 0.5794 - val_loss: 0.9

KeyboardInterrupt: 

In [ ]:

scores = model.evaluate(x_test, y_test, batch_size = 16)


In [ ]:

model.save('./transfer_model_20.h5')